In [49]:
# import from DistanceMatrix
from ipynb.fs.full.DistanceMatrix_v2 import *

In [50]:
import pandas as pd

# get requests
requests = pd.read_csv('Simulated_Requests.csv', index_col = 0)
requests

,requests,time
G6,2,4
R2,2,4
G2,2,4
E7,1,2
G3,3,6
C9,1,2
C8,2,4
G11,2,4
F3,2,4
C3,2,4


In [51]:
# split the solution list by teams
def team_split(solution):
    split_indices = [i for i in range(len(solution)) if solution[i] == 'H1']
    split_solution = [solution[i:j] for i, j in zip(split_indices, split_indices[1:] + [None])]
    if len(split_solution) == 1:
        return split_solution[0]
    else:
        return split_solution

In [52]:
# objective function parameters
# r, alpha, beta, k = 10, 1, 1, 20

# elements in the objective function
def reward(solution, r = 10):
    reward = 0
    for i in range(len(solution)):
        if solution[i] != 'H1':
            reward += r * requests.loc[solution[i], 'requests']
    return reward

def travel_time_cost(solution, alpha = 1):
    travel_time_cost = 0
    for i in range(len(solution)):
        if solution[i] != 'H1':
            travel_time_cost += alpha * get_shortest_path_length(graph, solution[i-1], solution[i])
    return travel_time_cost

def uneven_caseload_cost(solution, beta = 1):
    uneven_caseload_cost = 0
    total_caseload = 0
    for i in solution:
        if i != 'H1':
            total_caseload += requests.loc[i, 'requests']
    mean_caseload = total_caseload / len(team_split(solution))
    for j in team_split(solution):
        caseload = 0
        for k in j:
            if k != 'H1':
                caseload += requests.loc[k, 'requests']
        uneven_caseload_cost += beta * abs(caseload - mean_caseload)
    return uneven_caseload_cost

def overtime_cost(solution, k = 20, shift_time = 120, max_overtime = 15):
    overtime_cost = 0
    for i in team_split(solution):
        total_time = 0
        for j in range(len(i)):
            if i[j] != 'H1':
                total_time += get_shortest_path_length(graph, i[j-1], i[j])
                total_time += requests.loc[i[j], 'time']
        if (total_time - shift_time) <= max_overtime:
            overtime_cost += 0
        else:
            overtime_cost += k * (total_time - shift_time - max_overtime) ** 2
    return overtime_cost

# objective function
def objective_function(solution):
    objective_function = reward(solution)
    objective_function -= travel_time_cost(solution)
    objective_function -= uneven_caseload_cost(solution)
    objective_function -= overtime_cost(solution)
    return objective_function

In [53]:
# hard constraint
# route is eligible if it does not exceed shift time
# parameter solution is the route of one team
# paramter shift is the shift time for one team
# def shift_time_constraint(solution, shift):
#     time = 0
#     for i in range(len(solution) - 1):
#         time += get_shortest_path_length(graph, solution[i], solution[i+1])
#         time += requests.loc[solution[i], 'time']
#     if time <= shift:
#         return True
#     else:
#         return False

In [54]:
import random

# randomly assign wards to teams & not exceed shift time constraint
# parameter requests is the dataframe of requests
# paramter team is the number of teams
def get_InitialSolution(requests, team, shift_time = 120, max_overtime = 15):
    initial_solution = []
    unvisited = list(requests.index.values)
    for i in range(team):
        total_time = 0
        initial_solution_team = []
        initial_solution_team.append('H1') # team starts at ward H1
        while (total_time - shift_time) <= max_overtime: # not exceed maximum overtime
#        while total_time < shift_time: # not exceed shift time
            if len(unvisited) == 0:
                break
            ward = random.choice(unvisited)
            total_time += get_shortest_path_length(graph, ward, initial_solution_team[-1])
            total_time += requests.loc[ward, 'time']
            initial_solution_team.append(ward)
            unvisited.remove(ward)
        initial_solution += initial_solution_team
    return initial_solution

In [55]:
# swap the location of two wards in the solution
# def swap_operator(solution):
#     neighbors = []
#     for i in range(len(solution)):
#         for j in range(i+1, len(solution)):
#             if solution[i] != 'H1' and solution[j] != 'H1':
#                 neighbor = solution[:]
#                 neighbor[i], neighbor[j] = neighbor[j], neighbor[i] # swap the location
#                 neighbors.append(neighbor)
#     return neighbors

In [56]:
# randomly swap the location of two wards in the location
# improve speed - not search thoroughly
def swap_operator(solution, max_neigh):
    neighbors = []
    while len(neighbors) < max_neigh:
        i = random.choice(range(len(solution)))
        j = random.choice(range(len(solution)))
        if i != j and solution[i] != 'H1' and solution[j] != 'H1':
            neighbor = solution[:]
            neighbor[i], neighbor[j] = solution[j], solution[i]
            try:
                neighbors.index(neighbor)
            except:
                neighbors.append(neighbor)
    return neighbors

In [57]:
# change the location of a ward in the solution
# def relocate_operator(solution):
#     neighbors = []
#     for i in range(len(solution)):
#         if solution[i] != 'H1':
#             neighbor = solution[:]
#             ward = neighbor[i]
#             neighbor.pop(i)
#             for j in range(len(neighbor)):
#                 if j != i and neighbor[j] != 'H1':
#                     neighbor2 = neighbor.copy()
#                     neighbor2.insert(j, ward)
#                     neighbors.append(neighbor2)
#     return neighbors

In [58]:
# change the location of a random ward in the solution
# improve speed - not search thoroughly
def relocate_operator(solution, max_neigh):
    neighbors = []
    while len(neighbors) < max_neigh:
        i = random.choice(range(len(solution)))
        if solution[i] != 'H1':
            neighbor = solution[:]
            ward = neighbor[i]
            neighbor.pop(i)
            j = random.choice(range(len(neighbor)))
            if neighbor[j] != i and neighbor[j] != 'H1':
                neighbor.insert(j, ward)
                try:
                    neighbors.index(neighbor)
                except:
                    neighbors.append(neighbor)
    return neighbors

In [59]:
# insert a new ward to the solution
# def insert_operator(solution, requests):
#     neighbors = []
#     unvisited = list(requests.index.values)
#     for i in range(len(solution)):
#         if solution[i] != 'H1':
#             unvisited.remove(solution[i]) 
#         # create a list that contains all the wards not visited
#     for j in range(len(solution)):
#         if solution[j] != 'H1':
#             for k in range(len(unvisited)):
#                 neighbor = solution[:]
#                 neighbor.insert(j, unvisited[k])
#                 neighbors.append(neighbor)
#     return neighbors

In [60]:
# insert a random new ward to a random location in the solution
# improve speed - not search thoroughly
def insert_operator(solution, requests, max_neigh):
    neighbors = []
    unvisited = list(requests.index.values)
    for i in range(len(solution)):
        if solution[i] != 'H1':
            unvisited.remove(solution[i]) 
        # create a list that contains all the wards not visited
    while len(neighbors) < max_neigh:
        j = random.choice(range(len(solution)))
        if solution[j] != 'H1':
            k = random.choice(unvisited)
            neighbor = solution[:]
            neighbor.insert(j, k)
            try:
                neighbors.index(neighbor)
            except:
                neighbors.append(neighbor)
    return neighbors

In [61]:
# remove a ward from the solution and add a new ward to the solution
# def change_operator(solution, requests):
#     neighbors = []
#     unvisited = list(requests.index.values)
#     for i in range(len(solution)):
#         if solution[i] != 'H1':
#             unvisited.remove(solution[i]) # create a list that contains all the wards not visited
#     for j in range(len(solution)):
#         if solution[j] != 'H1':
#             for k in range(len(unvisited)):
#                 neighbor = solution[:]
#                 neighbor[j] = unvisited[k] # remove a ward and add a new ward
#                 neighbors.append(neighbor)
#     return neighbors

In [62]:
# remove a ward from the solution and add a new ward to the solution
# improve speed - not search thoroughly
def change_operator(solution, requests, max_neigh):
    neighbors = []
    unvisited = list(requests.index.values)
    for i in range(len(solution)):
        if solution[i] != 'H1':
            unvisited.remove(solution[i]) # create a list that contains all the wards not visited
    while len(neighbors) < max_neigh:
        j = random.choice(range(len(solution)))
        if solution[j] != 'H1':
            k = random.choice(unvisited)
            neighbor = solution[:]
            neighbor[j] = k
            try:
                neighbors.index(neighbor)
            except:
                neighbors.append(neighbor)
    return neighbors

In [63]:
# use move operators to get neighbors
def get_neighbors(solution, max_neigh = 10):
    neighbors = []
    unvisited = list(requests.index.values)
    for i in range(len(solution)):
        if solution[i] != 'H1':
            unvisited.remove(solution[i]) # create a list that contains all the wards not visited
    if len(unvisited) == 0:
        neighbors += swap_operator(solution, max_neigh)
        neighbors += relocate_operator(solution, max_neigh) 
        # perform only swap & relocate operators if no ward is left
    else:
        neighbors += swap_operator(solution, max_neigh)
        neighbors += relocate_operator(solution, max_neigh)
        neighbors += change_operator(solution, requests, max_neigh)
        neighbors += insert_operator(solution, requests, max_neigh)
        # perform all operators if there are wards left
    return neighbors

In [64]:
# alternative way to get neighbors
# def get_neighbors(solution, max_neigh = 999999):
#     neighbors = []
#     unvisited = list(requests.index.values)
#     for i in range(len(solution)):
#         if solution[i] != 'H1':
#             unvisited.remove(solution[i]) # create a list that contains all the wards not visited
#     if len(unvisited) == 0:
#         rand = random.randint(0, 1)
#         if rand == 0:
#             neighbors += swap_operator(solution, max_neigh)
#         else:
#             neighbors += relocate_operator(solution, max_neigh)
#         # randomly perform swap or relocate if no ward is left
#     else:
#         rand = random.randint(0, 3)
#         if rand == 0:
#             neighbors += swap_operator(solution, max_neigh)
#         elif rand == 1:
#             neighbors += relocate_operator(solution, max_neigh)
#         elif rand == 2:
#             neighbors += change_operator(solution, requests, max_neigh)
#         else:
#             neighbors += insert_operator(solution, requests, max_neigh)
#         # randomly perform swap, relocate, change, or insert
#     return neighbors

In [65]:
# def tabu_search(initial_solution, tabu_list_size, max_iterations):
#     current_solution = initial_solution
#     best_solution = initial_solution
#     tabu_list = []
#     for i in range(max_iterations):
#         neighbors = get_neighbors(current_solution)
#         non_tabu_neighbors = [n for n in neighbors if n not in tabu_list]
#         if len(non_tabu_neighbors) == 0:
#             break
#         non_tabu_neighbors.sort(key = objective_function, reverse = True)
#         next_solution = non_tabu_neighbors[0]
#         tabu_list.append(next_solution)
#         if len(tabu_list) > tabu_list_size:
#             tabu_list.pop(0)
#         current_solution = next_solution
#         if objective_function(current_solution) > objective_function(best_solution):
#             best_solution = current_solution
#     return best_solution

In [82]:
import time

def tabu_search(initial_solution, tabu_list_size, max_iterations, max_neigh):
    start = time.time()
    solution = {} # return this dictionary as result
    current_solution = initial_solution
    best_solution = initial_solution
    tabu_list = []
    for i in range(max_iterations):
        neighbors = get_neighbors(current_solution, max_neigh = max_neigh)
        neighbors.sort(key = objective_function, reverse = True)
        non_tabu_neighbors = [n for n in neighbors if n not in tabu_list]
        count = 0
        while count <= len(neighbors):
            if objective_function(neighbors[0]) > objective_function(best_solution):
                current_solution = neighbors[0]
                best_solution = neighbors[0]
                break
            elif neighbors[count] in non_tabu_neighbors:
                current_solution = neighbors[count]
                break
            else:
                count += 1
#         else:
#             non_tabu_neighbors = [n for n in neighbors if n not in tabu_list]
#             if len(non_tabu_neighbors) == 0:
#                 break
#             non_tabu_neighbors.sort(key = objective_function, reverse = True)
#             current_solution = non_tabu_neighbors[0]
#             if objective_function(current_solution) > objective_function(best_solution):
#                 best_solution = current_solution
        tabu_list.append(current_solution)
        if len(tabu_list) > tabu_list_size:
            tabu_list.pop(0)
    end = time.time()
    comp_time = round(end - start) # computation time in seconds
    best_split = team_split(best_solution)
    route = [] # route for each team
    total_time = [] # total time spent for each team
    travel_time = [] # travel time spent for each team
    service_time = [] # service time spent for each team
    n_requests_finished = [] # number of requests finsihed for each team
    for i in best_split:
        route.append(i)
        travel_time_team = 0
        service_time_team = 0
        n_requests_finished_team = 0
        for j in range(1, len(i)):
            travel_time_team += get_shortest_path_length(graph, i[j-1], i[j])
            service_time_team += requests.loc[i[j], 'time']
            n_requests_finished_team += requests.loc[i[j], 'requests']
        total_time_team = travel_time_team + service_time_team
        total_time.append(total_time_team)
        travel_time.append(travel_time_team)
        service_time.append(service_time_team)
        n_requests_finished.append(n_requests_finished_team)
    solution['route'] = route
    solution['total_time'] = total_time
    solution['travel_time'] = travel_time
    solution['service_time'] = service_time
    solution['n_requests_finished'] = n_requests_finished
    solution['obj_value'] = objective_function(best_solution)
    solution['comp_time'] = comp_time
    return solution

In [67]:
initial = get_InitialSolution(requests, 5)
print(initial)
objective_function(initial)

['H1', 'C2', 'R9', 'H10', 'H1', 'R10', 'C3', 'D5', 'H1', 'M5', 'F9', 'ALG', 'B7', 'R3', 'H1', 'G5', 'G6', 'F3', 'A8', 'H1', 'D3', 'E5', 'H9', 'C6']


-86991.6

In [83]:
solution = tabu_search(initial, 100, 500, 25)
print(solution)

{'route': [['H1', 'H6', 'C6', 'D6', 'A6', 'B6'], ['H1', 'G7', 'H7', 'E7', 'F10', 'G10'], ['H1', 'A1', 'B9', 'C9', 'B10', 'A10', 'A11', 'C11'], ['H1', 'C7', 'A7', 'C3', 'E3', 'H3'], ['H1', 'C8', 'C4', 'E4', 'H4']], 'total_time': [132.0, 135.0, 133.0, 129.0, 131.0], 'travel_time': [44.0, 55.0, 71.0, 73.0, 67.0], 'service_time': [88, 80, 62, 56, 64], 'n_requests_finished': [44, 40, 31, 28, 32], 'obj_value': 1412.0, 'comp_time': 301}


# tune hyper-parameter

In [21]:
# hyper_parameter = {'tabu_list_size': [50, 100, 150],
#                   'max_iterations': [500],
#                   'max_neigh': [20, 25, 30]}

In [22]:
# solutions = {}

# for i in range(50):
#     tabu_list_size = random.choice(hyper_parameter['tabu_list_size'])
#     max_iterations = hyper_parameter['max_iterations'][0]
#     max_neigh = random.choice(hyper_parameter['max_neigh'])
#     solution = tabu_search(initial, tabu_list_size, max_iterations, max_neigh)
#     solutions[i] = solution

# Comparison

In [84]:
import pandas as pd

# get requests
requests = pd.read_csv('sub20230114_agg.csv', index_col = 0)
requests['time'] = requests['Number of Scans'] * 2
requests = requests.rename(columns = {"Number of Scans": "requests"})

In [85]:
requests

,requests,time
Ward,,
A1,3,6
A10,5,10
A11,1,2
A3,5,10
A4,5,10
A5,8,16
A6,14,28
A7,5,10
A8,6,12


In [87]:
initial = get_InitialSolution(requests, 5)
print(initial)
objective_function(initial)

['H1', 'B6', 'C3', 'H10', 'G10', 'H1', 'E9', 'C8', 'H6', 'H1', 'E2', 'F10', 'H8', 'H3', 'H1', 'E8', 'E7', 'B5', 'E3', 'H1', 'G4', 'G7', 'C7', 'A7']


-13970.2

In [88]:
values = [10, 20, 30, 50, 100, 150, 200, 250, 300, 400, 500, 600, 700]
obj_value = []
comp_time = []

for i in values:
    solution = tabu_search(initial, 100, i, 25)
    obj_value.append(solution['obj_value'])
    comp_time.append(solution['comp_time'])

In [89]:
obj_value

[875.8,
 1057.6,
 1180.4,
 1193.8,
 1352.0,
 1483.0,
 1363.0,
 1372.0,
 1456.2,
 1343.8,
 1220.0,
 1358.8,
 1386.6]

In [90]:
comp_time

[6, 12, 16, 28, 55, 77, 118, 157, 175, 239, 328, 412, 456]

In [78]:
values = [10, 10, 10, 20, 50, 50, 50, 50, 50, 100, 100, 100, 100]
obj_value = []
comp_time = []

for i in values:
    solution = tabu_search(initial, 100, i, 25)
    obj_value.append(solution['obj_value'])
    comp_time.append(solution['comp_time'])
    initial = solution['route'][0] + solution['route'][1] + solution['route'][2] + solution['route'][3] + solution['route'][4]

In [79]:
obj_value

[951.4,
 1203.8,
 1243.0,
 1315.4,
 1393.2,
 1397.2,
 1397.2,
 1410.0,
 1412.0,
 1412.0,
 1412.0,
 1412.0,
 1412.0]

In [80]:
comp_time

[7, 7, 11, 23, 55, 57, 58, 60, 58, 133, 116, 104, 103]